In [2]:
import pandas as pd

In [3]:
df_2020= pd.read_csv (r'full_2020.csv')
df_2019 = pd.read_csv (r'full_2019.csv')
df_2018 = pd.read_csv (r'full_2018.csv')

In [9]:
full.head()

,Spot,date,time,range,height_ft,swell_dir,swell_angle,period_sec,wind_kmh,wind_dir,wind,wind_angle,temp_°c,Conditions
0,Nazare,2018-01-01,6am,20-30ft,15.5,WNW,292,15,6,S,Cross/Offshore,186,8,Good
1,Nazare,2018-01-01,12pm,18-28ft,14.5,WNW,292,15,14,W,Onshore,277,15,Good
2,Nazare,2018-01-01,6pm,18-26ft,13.5,WNW,293,14,6,WSW,Onshore,253,11,Good
3,Nazare,2018-02-01,6am,16-26ft,12.5,WNW,297,14,5,SSW,Cross-shore,207,10,Good
4,Nazare,2018-02-01,12pm,14-24ft,11.5,WNW,297,14,11,WSW,Onshore,258,16,Good


In [4]:
full = pd.concat([df_2018,df_2019,df_2020], ignore_index=True)

In [10]:
full["date"] = pd.to_datetime(full.date)

In [ ]:
full.ty